In [1]:
import torch
import torch.nn as nn
import os
import torch.utils.data as data
from importlib import import_module

vggish_input=import_module(".torchvggish.vggish_input","torchvggish-master")


#GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
#損失関数
criterion = nn.CrossEntropyLoss()


vggish/embedding:0
Using cuda device


# 今日は試しに学習させてみる


In [2]:
model = torch.hub.load('torchvggish-master', 'vggish', source='local', preprocess=False).to(device)
#harritaylor/
#model.eval()
'''
# Download an example audio file
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

model.forward(filename)```'''

vggish/embedding:0


'\n# Download an example audio file\nimport urllib\nurl, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")\ntry: urllib.URLopener().retrieve(url, filename)\nexcept: urllib.request.urlretrieve(url, filename)\n\nmodel.forward(filename)```'

In [3]:
class MyClassfilter(nn.Module):
    def __init__(self):
        super(MyClassfilter, self).__init__()
        self.classfilter=nn.Sequential(
            nn.Linear(128,64),
            nn.Sigmoid(),
            nn.Linear(64,5),
            nn.Softmax()
        )
    def forward(self, x):
        return self.classfilter(x)
mymodel = MyClassfilter().to(device)
mymodel.eval()

MyClassfilter(
  (classfilter): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=64, out_features=5, bias=True)
    (3): Softmax(dim=None)
  )
)

In [4]:
# 学習済みの重みを使用
use_pretrained = True

# モデルをロード
print(model)

VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [5]:
#print('変更前 : ', model.embeddings[4])
#model.embeddings[4] = nn.Linear(in_features=4096, out_features=5)
#print('変更あと : ', model.embeddings[4])

In [6]:
#model.classfilter=nn.Sequential(
#    nn.Linear(128, 5)   
#)
#print(model)

In [7]:
#試しに実行
#model.eval()# 評価しない
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
#a =model.forward(filename)
#rint(a.size(),a.dim())
#print(mymodel.forward(a))

In [8]:
#データセットの定義？
classes=["Speech","Wind","Animal","Music","Speech"]
dataset_dir="C:/Users/hase/git/youtubedl/result"


def make_path_list():
    train_file_list=[]
    valid_file_list=[]
    for i in range(len(classes)):
        dir_name=os.path.join(dataset_dir,classes[i]).replace("\\","/")
        file_list=os.listdir(dir_name)
        
        #8割を学習用、残りを検証用にする
        num_data = len(file_list)
        num_split = int(num_data*0.8)
        
        train_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[:num_split] ]
        valid_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[num_split:]]
    return {"train":train_file_list,"valid":valid_file_list}
# 画像データへのファイルパスとラベルを格納したリストを取得する
data_dict = make_path_list()
## リストが変かもなのでみてみる


print('学習データ数 : ', len(data_dict["train"]))
##### 先頭3件だけ表示
print(data_dict["train"][:3])

print('検証データ数 : ', len(data_dict["valid"]))
##### 先頭3件だけ表示
print(data_dict["valid"][:3])

学習データ数 :  162
[['C:/Users/hase/git/youtubedl/result/Speech/--aE2O5G5WE.wav', 0], ['C:/Users/hase/git/youtubedl/result/Speech/--PJHxphWEs.wav', 0], ['C:/Users/hase/git/youtubedl/result/Speech/-30H9V1IKps.wav', 0]]
検証データ数 :  44
[['C:/Users/hase/git/youtubedl/result/Speech/-L44JqysUy8.wav', 0], ['C:/Users/hase/git/youtubedl/result/Speech/-L68XDzhn2g.wav', 0], ['C:/Users/hase/git/youtubedl/result/Speech/-LBl0UeKNyU.wav', 0]]


In [9]:
#0830
#DataSet型を作る必要がある気がする
class MyDataset(data.Dataset):
    '''
    data_dictは[パス,番号]
    '''
    def __init__(self, _data_dict,  phase='train'):
        #self.data_dict = data_dict
        self.data_dict = []
        for path, label in _data_dict:
            for data in vggish_input.wavfile_to_examples(path):
                self.data_dict.append([data, label])
                
        
        self.phase = phase
        
    def __len__(self):
        return len(self.data_dict)
    
    def __getitem__(self,index):
        
        wav_path,label = self.data_dict[index]
        
        return wav_path, label

#DataSetを実際に作ってみる 

train_dataset = MyDataset(
    _data_dict=data_dict["train"],
    phase="train"
)

valid_dataset = MyDataset(
    _data_dict=data_dict["valid"],
    phase="valid"
)




In [10]:
#test
tst=train_dataset.__getitem__(0)
print(tst,tst[0].size())

(tensor([[[-4.6052, -4.6052, -4.6052,  ..., -4.6052, -4.6052, -4.6052],
         [-4.6052, -4.6052, -4.6052,  ..., -4.6052, -4.6052, -4.6052],
         [-4.6047, -4.6047, -4.6046,  ..., -4.6019, -4.6023, -4.6025],
         ...,
         [ 2.8631,  2.4165,  1.0068,  ..., -1.3567, -0.7359, -0.9956],
         [ 2.4454,  2.8380,  2.3015,  ..., -1.1201, -1.2774, -0.9788],
         [ 2.8262,  2.5614,  1.9133,  ..., -1.2405, -1.3049, -1.1985]]],
       grad_fn=<UnbindBackward>), 0) torch.Size([1, 96, 64])


In [11]:
#0830
#もう1手ひつようらしい

#dataloaderを用いてミニバッチを作成
batch_size=10

train_dataloader=data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)
valid_dataloader=data.DataLoader(
    valid_dataset, batch_size = batch_size//2, shuffle=True
)

dataloader_dict={
    'train': train_dataloader, 
    'valid': valid_dataloader
}
print(dataloader_dict)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x00000238239EB610>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x00000238239EBA30>}


In [12]:
# パラメータ名の確認
for name, param in model.named_parameters():
    print('name : ', name)
for prm in mymodel.named_parameters():
    print(prm)
#print(mymodel.parameters())

name :  features.0.weight
name :  features.0.bias
name :  features.3.weight
name :  features.3.bias
name :  features.6.weight
name :  features.6.bias
name :  features.8.weight
name :  features.8.bias
name :  features.11.weight
name :  features.11.bias
name :  features.13.weight
name :  features.13.bias
name :  embeddings.0.weight
name :  embeddings.0.bias
name :  embeddings.2.weight
name :  embeddings.2.bias
name :  embeddings.4.weight
name :  embeddings.4.bias
name :  pproc.pca_eigen_vectors
name :  pproc.pca_means
('classfilter.0.weight', Parameter containing:
tensor([[ 6.5616e-02,  5.6828e-02,  7.1750e-02,  ..., -1.2929e-02,
         -6.4636e-02, -8.6721e-02],
        [ 5.3197e-05,  3.2198e-02,  3.7424e-02,  ...,  5.2885e-02,
         -7.6512e-02, -3.9736e-02],
        [-1.5083e-02,  1.7054e-02,  8.1912e-02,  ..., -9.5320e-03,
         -7.7025e-03, -5.1590e-02],
        ...,
        [ 3.2359e-02,  8.7688e-02,  4.5022e-02,  ..., -2.2515e-02,
          4.7380e-02, -2.8570e-02],
      

In [13]:
#学習させるパラメータを格納
params_to_update=[]
#学習させるパラメータ名
update_param_names=['classfilter.0.weight', 'classfilter.0.bias', 'classfilter.2.weight']

#対象以外は購買計算をせず、変化しないようにもする
for name,param in mymodel.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print("name : ",name)
    else:
        param.requires_grad = False


name :  classfilter.0.weight
name :  classfilter.0.bias
name :  classfilter.2.weight


In [14]:
import torch.optim as optim
#optimizer = optim.SGD(params_to_update, lr=0.01)
optimizer = optim.SGD(params_to_update, lr=0.01)

In [16]:
#エポック数
num_epochs=20

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1,num_epochs))
    
    for phase in ['train','valid']:
        if phase == 'train':
            mymodel.train()#学習モード
        else:
            mymodel.eval()#検証？
    
        #epoch全体の損失の輪と正解数
        epoch_loss=0.0
        epoch_corrects=0
        
        count=0.0
        #print(phase)
        for inputs, labels in dataloader_dict[phase]:
            #入力の確認
            #print(len(inputs),len(labels))
            #optimizer?を初期化?する?
            optimizer.zero_grad()


            with torch.set_grad_enabled(phase=='train'):

                #outputs=mymodel(torch.tensor([model(input).tolist()[0] for input in inputs]).to(device))
               #0906 これ([0])はあかんやろってなった
                #for i, label in zip(inputs, labels):
                out1=model(inputs)
                outputs=mymodel(out1)
                    #print(output)
                    #for h in output:
                       # print(h.tolist())
                        #output_models.append(h.tolist())
                        #output_models.append(h)
                        #output_labels.append(label.tolist())
                        #count+=1
                        #print(label)
                
                #output_models=torch.tensor(output_models, device=device, grad_fn=output.grad_fn)
                #output_labels=torch.tensor(output_labels, device=device)
                labels=labels.to(device)
                #print(output_models)
                #loss=criterion(output_models, output_labels)
                loss=criterion(outputs, labels)
                #ラベルを予測
                _,preds = torch.max(outputs,1)
                
                #訓練時は逆伝搬の計算
                if phase == "train":
                    #逆伝搬
                    loss.backward()
                    
                    #パラメータ更新
                    optimizer.step()
                
                #イテレーション結果の計算
                #lossの合計を更新
                #pytorchの使用上バッチ内の平均lossが計算されているのでデータ数をかけて合計にする
                #損失和を「全データの損失/データ数」で求めるせいらしい?
                #print(len(inputs))
                epoch_loss += loss.item() * inputs.size(0)
                #epoch_loss += loss.item()*output_labels.size(0)
                
                #正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)
                #print(preds.size(),count,output_labels.size(0))
                #print()
                #epoch_corrects += torch.sum(preds == output_labels)
        #epochのlossと正解数の表示
        epoch_loss=epoch_loss/len(dataloader_dict[phase])
        epoch_acc=epoch_corrects.double()/len(dataloader_dict[phase].dataset)
        #print(epoch_loss, epoch_corrects.double(), count)
        #epoch_loss=epoch_loss/count
        #epoch_acc=epoch_corrects.double()/count
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        
    

Epoch 1/20
train Loss: 16.0135 Acc: 0.2407
valid Loss: 7.9769 Acc: 0.2528
Epoch 2/20
train Loss: 15.8836 Acc: 0.2886
valid Loss: 7.9608 Acc: 0.2323
Epoch 3/20
train Loss: 15.7539 Acc: 0.2600
valid Loss: 7.8947 Acc: 0.2916
Epoch 4/20
train Loss: 15.5667 Acc: 0.3638
valid Loss: 7.8004 Acc: 0.2825
Epoch 5/20
train Loss: 15.3726 Acc: 0.3713
valid Loss: 7.8019 Acc: 0.2597
Epoch 6/20
train Loss: 15.2257 Acc: 0.4229
valid Loss: 7.7344 Acc: 0.3622
Epoch 7/20
train Loss: 15.0722 Acc: 0.4415
valid Loss: 7.6456 Acc: 0.4032
Epoch 8/20
train Loss: 14.8936 Acc: 0.4677
valid Loss: 7.5689 Acc: 0.3759
Epoch 9/20
train Loss: 14.7809 Acc: 0.4863
valid Loss: 7.5361 Acc: 0.4191
Epoch 10/20
train Loss: 14.6547 Acc: 0.5100
valid Loss: 7.4905 Acc: 0.4055
Epoch 11/20
train Loss: 14.5251 Acc: 0.5000
valid Loss: 7.4481 Acc: 0.4647
Epoch 12/20
train Loss: 14.4628 Acc: 0.5187
valid Loss: 7.3934 Acc: 0.4579
Epoch 13/20
train Loss: 14.2824 Acc: 0.5336
valid Loss: 7.4353 Acc: 0.4396
Epoch 14/20
train Loss: 14.1687 Ac

In [ ]:









m = nn.Softmax(dim=2)
input = torch.randn(4, 4)
output = m(input)
print(input)
print(output)

In [ ]:
m = nn.Sigmoid()
input = torch.randn(2)
output = m(input)
print(input,output)

In [ ]:
model.forward("C:/Users/hase/git/youtubedl/result/Speech/--aE2O5G5WE.wav")

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)

output = loss(input, target)
output.backward()
print(input,target,output, sep='\n')
print(output.grad_fn)

In [ ]:
a=torch.tensor([])
a

In [ ]:
a.append([1])

In [ ]:
tst= vggish_input.wavfile_to_examples("C:/Users/hase/git/youtubedl/result/Speech/--aE2O5G5WE.wav")
print(tst.size())
for i in tst:
    print(i.size())
    print (model(i))
